In [1]:
from libs.SatelliteEnvironment import *
from libs.DRLAgents.DQN import DQN
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import torch 

C:\Users\minstone\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
env = CircularOrbit()

state = env.get_state()

num_states = state.shape[0]
orbit_num  = env.orbit_num
sat_num    = env.satellite_num
num_action = orbit_num * sat_num

model      = DQN(num_states = num_states,
                 num_action = num_action)

writer     = SummaryWriter('runs/satellite')
episodes   = 10000

for i in range(episodes):
    env.reset()
    ep_reward = 0
    ep_loss = 0 
    while True:
        state = env.get_state()
        
        action = model.choose_action(state)

        reward, done, info = env.step(action) # 
        
        next_state = env.get_state() # 
        
        model.store_transition(state, action, reward, next_state) # 
        ep_reward += reward

        if model.memory_counter >= MEMORY_CAPACITY:

            loss = model.learn(network_iteration= 100,
                                batch_size= 32,
                                gamma=0.9)
            ep_loss += loss
            if done:
                print("episode: {} , the episode reward is {}".format(i, round(ep_reward, 3)))
        if done:
            break

    writer.add_scalar('Episode Reward', ep_reward, i)
    writer.add_scalar('Episode Loss', ep_loss, i)

writer.close()

108.91342261475782
337.01270630537624
108.91342261475782
121.82019183023519
123.99433452460164
108.91342261475782
108.91342261475782
108.91342261475782
108.91342261475782
108.91342261475782
108.91342261475782
108.91342261475782
108.91342261475782
337.01270630537624
108.91342261475782
108.91342261475782
108.91342261475782
108.91342261475782
108.91342261475782
108.91342261475782
108.91342261475782
121.82019183023519
123.99433452460164
108.91342261475782
4.827790596139948
-5.287554837937847


ValueError: math domain error